### Powershell 또는 Terminal 으로 가상 환경 생성

In [44]:
!pip install --quiet pandas openpyxl python-dotenv google-generativeai
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


### 라이브러리 실행

In [45]:
# 라이브러리
import io
from typing import Any, Dict, List
import tqdm as notebook_tqdm

import pandas as pd
import google.generativeai as genai
from dotenv import load_dotenv
from pathlib import Path
import os

### API key 등록 후 모델 버전 설정

In [ ]:
api_key = "API_KEY_HERE"
genai.configure(api_key=api_key)

# Pick a Gemini model name
GEMINI_MODEL = "gemini-2.5-pro"

### 요약 기능에 필요한 엑셀 함수

In [47]:
def _summarize_single_sheet(df: pd.DataFrame, sheet_name: str) -> Dict[str, Any]:
    """Extract simple stats to send to the LLM."""
    info: Dict[str, Any] = {
        "sheet_name": sheet_name,
        "n_rows": int(len(df)),
        "n_cols": int(len(df.columns)),
        "columns": [],
        "preview": df.head(5).to_dict(orient="records"),
    }

    for col in df.columns:
        s = df[col]
        col_info: Dict[str, Any] = {
            "name": str(col),
            "dtype": str(s.dtype),
            "null_count": int(s.isna().sum()),
        }
        if pd.api.types.is_numeric_dtype(s):
            if s.notna().any():
                col_info["min"] = float(s.min(skipna=True))
                col_info["max"] = float(s.max(skipna=True))
                col_info["mean"] = float(s.mean(skipna=True))
            else:
                col_info["min"] = col_info["max"] = col_info["mean"] = None
        info["columns"].append(col_info)

    return info


def analyze_excel_bytes(content: bytes) -> Dict[str, Any]:
    """Parse Excel bytes to minimal numeric/text structure for LLM summarization."""
    excel = pd.ExcelFile(io.BytesIO(content))

    sheets_meta: List[Dict[str, Any]] = []
    for sheet_name in excel.sheet_names:
        df = excel.parse(sheet_name)
        sheets_meta.append(_summarize_single_sheet(df, sheet_name))

    return {"sheets": sheets_meta}

### 프롬프트 엔지니어링 함수
- system_prompt 에서 지침 입력 
- temperature 으로 정보 기반 분석 설정 (0.1 ~ 1.0)

In [48]:
def llm_summarize_excel(structured: Dict[str, Any]) -> str:
    """
    Call Gemini to turn the structured info into a natural-language summary.
    """
    system_prompt = (
        """You are a senior Korean data analyst writing a professional corporate report.

        The user uploads an Excel file. You receive extracted metadata for each sheet,
        including columns, null counts, and a preview of several rows.

        IMPORTANT:
        - The Excel file may contain many blank columns, header-like rows, title rows,
        or explanatory text before the actual data.
        - Your job is to infer the TRUE data structure and meaning, even if the first rows
        contain titles, notes, or empty cells.
        - It is very important to mention information in this format: ABC column has values like 123, 456, 789.
        - It is very important to mention information in this format: ABC row has values like 123, 456, 789.
        - Ignore rows that are titles, subtitles, blank rows, or "header text" such as
        reports notes, system messages, or explanations.
        - Do NOT mention that information is unavailable or missing.

        Write a clean, professional summary in **Korean**, in paragraph form
        (not bullet points, not markdown).

        Your summary MUST include:
        1. What the dataset represents (the underlying real-world meaning).
        2. Key metrics, meaningful columns, and what they measure.
        3. Visible numerical trends such as 증가/감소, 큰 값/작은 값, 범위, 이상치.
        4. Business insights that can be inferred from the available numbers.
        5. A concise concluding interpretation.
        6. First sentence should include numerical statement of the table and second sentence should inlcude description of conclusion.
        7. Only output 2 sentences.
        8. End the sentences with phrases like "으로 나타남", "로 분석됨", "로 확인됨".

        DO NOT:
        - Do NOT mention sheet names.
        - Do NOT list blank columns or columns named “Unnamed”.
        - Do NOT describe metadata (“null_count”, “n_rows”).
        - Do NOT reference rows that contain no meaningful data.
        - Do NOT include markdown such as ### or **bold**.
        - Do NOT mention that information is unavailable or missing.
        - Do NOT include words like "집단" or "조직"
        - Do NOT mention phrases like "분석한 결과" or "데이터에 따르면".

        Look at only 1 or 2 high numbers or values from the data, and mention them.
        
        This is an example of the type of report summary I want you to produce:
        업무기간 비중이 ‘3년 이상’22명(38%), ‘1년~2년’15명(26%)순으로 나타남
        """
    )

    user_content = (
        "Here is the structured description of the Excel file:\n\n"
        f"{structured}"
    )

    model = genai.GenerativeModel(GEMINI_MODEL)
    response = model.generate_content(
        [
            {"role": "user", "parts": [system_prompt + "\n\n" + user_content]}
        ],
        generation_config={
            "temperature": 0.3,
        },
    )

    # Gemini responses usually have .text
    return (response.text or "").strip()

### 한국어 요약 출력 함수

In [49]:
def summarize_excel_file(path: str) -> str:
    """
    Read an Excel file from disk, build structured meta,
    call Gemini, and return the Korean summary.
    """
    path_obj = Path(path)
    if not path_obj.exists():
        raise FileNotFoundError(f"Excel file not found: {path}")

    with path_obj.open("rb") as f:
        content = f.read()

    structured = analyze_excel_bytes(content)
    summary = llm_summarize_excel(structured)
    return summary

### 최종 출력 테스트

In [50]:
excel_path = "/Users/skeppy/Desktop/Excel_Test.xlsx"

summary_text = summarize_excel_file(excel_path)
print(summary_text)

성능 만족도 조사 결과, '만족' 응답이 34명으로 가장 높은 비중을 차지했으며 '매우 만족'이 12명으로 그 뒤를 잇는 것으로 나타남. 이는 대다수의 응답자가 현재 시스템 성능에 대해 긍정적으로 평가하고 있음을 의미하는 것으로 분석됨.
